In [32]:
from bs4 import BeautifulSoup
from requests import get
import csv
import re
import sqlite3

In [48]:
def price_format(price):
    return float(price.replace(" ", "").replace("zł", "").replace(",","."))


def pname_format(product):
    return product.replace(",",".")

def food_data(last_page_num=1):
    
    url = 'https://www.carrefour.pl/artykuly-spozywcze'
        
    global food_list
    food_list=[]
    
    global food_column_names
    food_column_names = ['Produkt','Cena','Cena za jednsotkę','Waga [KG]','Ilość [Sztuki]','Litry']
    
    
    for page_num in range(0,last_page_num):
        
        site = get(f'{url}?page={page_num}')
        bs = BeautifulSoup(site.content, 'html.parser')
        

        for offer in bs.find_all('div', class_='jss304'):

            try:

                product = pname_format(offer.find('span',class_='MuiTypography-root jss199 jss310 MuiTypography-h5').get_text().strip())

                price = price_format(offer.find('div', class_='MuiTypography-root jss318 MuiTypography-h3').get_text().strip())

                measure = offer.find('span',class_='MuiTypography-root jss199 jss321 MuiTypography-caption MuiTypography-colorTextSecondary').get_text().strip()

                if re.search(r'kg\Z', measure):
                    weight = float(re.split(r' kg\Z', measure)[0].replace(",","."))
                    quantity = 'N/D'
                    liters = 'N/D'

                elif re.search(r'szt.\Z', measure):
                    weight = 'N/D'
                    quantity = float(re.split(r' szt.\Z', measure)[0].replace(",","."))
                    liters = 'N/D'

                elif re.search(r'l\Z', measure):
                    weight = 'N/D'
                    quantity = 'N/D'
                    liters = float(re.split(r' l\Z', measure)[0].replace(",","."))


                price_kg_raw = offer.find('span',class_='MuiTypography-root MuiTypography-caption MuiTypography-colorTextSecondary').get_text().strip()


                if re.search(r'zł/1 kg\Z', price_kg_raw):
                    if re.search(r'\A.\xa0', price_kg_raw):
                        price_kg = float("".join(re.split(r'\xa0',re.split(r'zł/1 kg\Z', price_kg_raw)[0].replace(",","."))))
                    else:             
                        price_kg = float(re.split(r'zł/1 kg\Z', price_kg_raw)[0].replace(",","."))

                elif re.search(r'szt.\Z', price_kg_raw):
                    price_kg = float(re.split(r' zł/1 szt.\Z', price_kg_raw)[0].replace(",","."))

                elif re.search(r'l\Z', price_kg_raw):
                    price_kg = float(re.split(r' zł/1 l\Z', price_kg_raw)[0].replace(",","."))


                food_list.append([product,price,price_kg,weight,quantity,liters])

            except AttributeError:
                pass

def data_to_csv(csv_file_name='products', product_list=food_list):
    
    with open(f'{csv_file_name}.csv', 'w', encoding='utf-8', newline='') as products_csv:
        writer=csv.writer(products_csv)
        writer.writerow(column_names)
        writer.writerows(product_list)
        
def data_to_DB(DB_name='products', table_name='Products', product_list=food_list, truncate=True):
    
    
    con = sqlite3.connect(f'{DB_name}.db', timeout=15)
    
    cur = con.cursor()
    
    try:
        cur.execute(f'CREATE TABLE {table_name}{tuple(column_names)}')
    except:
        if truncate:
            cur.execute(f'DROP TABLE {table_name};')
            cur.execute(f'CREATE TABLE {table_name}{tuple(column_names)}')
        pass
        
    for product in product_list:
        cur.execute(f'INSERT INTO {table_name} VALUES {tuple(product)}')
    
    con.commit()
    con.close()

food_data(2)
data_to_DB()
            
        

OperationalError: database is locked